<a href="https://colab.research.google.com/github/eyamilabraham/draw-numbers-detect-numbers/blob/main/draw_numbers_detect_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

#Download MNIST dataset (handwritten numbers, labels)
data, metadata = tfds.load('mnist', as_supervised=True, with_info=True)

#Splitting variables of training (60k) and testing (10k)
data_training, data_testing = data['train'], data['test']

## labels of the 10 possible categories (simply the numbers from 0 to 9)
classes_names = metadata.features['label'].names

#Normalizing metadata (transforming from 0-255 to 0-1)
#The network learns better and faster
def normalize(pictures, labels):
  pictures = tf.cast(pictures, tf.float32)
  pictures /= 255 #Transforming from 0-255 to 0-1
  return pictures, labels

#Normalize metadata of training and testing with the above function
data_training = data_training.map(normalize)
data_testing = data_testing.map(normalize)

#Adding to cache (you have to use RAM in case of HDD or SSD, faster training)
data_training = data_training.cache()
data_testing = data_testing.cache()

#Creating the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax) #To neural networks of classification
])

#Compiling the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

#Splitting training and testing data (60k y 10k)
num_data_training = metadata.splits["train"].num_examples
num_data_testing = metadata.splits["test"].num_examples

#Batch work allows training with large amount of data to be done more efficiently
BATCH_SIZE = 32

#Shuffle y repeat data are randomly shuffled so that the network not learns the right way of the things
data_training = data_training.repeat().shuffle(num_data_training).batch(BATCH_SIZE)
data_testing = data_testing.batch(BATCH_SIZE)

import math

#Training
history = model.fit(data_training, epochs=5, steps_per_epoch= math.ceil(num_data_training/BATCH_SIZE))

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Epoch 1/5
1875/1875 [==============================] - 35s 8ms/step - loss: 0.3078 - accuracy: 0.9026
Epoch 2/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.1225 - accuracy: 0.9641
Epoch 3/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0918 - accuracy: 0.9735
Epoch 4/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0779 - accuracy: 0.9761
Epoch 5/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0654 - accuracy: 0.9812


In [2]:
#Exporting .h5 file
model.save('numbers.h5')

In [3]:
#You could use Linux commands if you first put '!' simbol. List the contents of the current folder to see that the model has been exported.
!ls

numbers.h5  sample_data


In [4]:
#To convert it to tensorflow.js, we must first install the library
!pip install tensorflowjs

     |████████████████████████████████| 77 kB 3.1 MB/s 


In [5]:
#Create a folder where the resulting files will be saved.
!mkdir output_folder

In [6]:
#Perform export to the 'output_folder'
!tensorflowjs_converter --input_format keras numbers.h5 output_folder/

In [7]:
#Confirm that the files have been generated in the 'output_folder'. There should be "bin" and "json" files.
!ls output_folder/

group1-shard1of1.bin  model.json


In [8]:
#To download them, left-click on the folder icon
#and expand 'output_folder'. In the files use the 3 dots to download them.